In [1]:
import random

def filter_sn_lightcurve_random(input_file, output_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    t0 = None
    header = []
    obs_lines = []
    footer = []

    # 1. Parse the file
    for line in lines:
        if line.startswith('PEAKMJD:'):
            t0 = float(line.split()[1])
        
        if line.startswith('OBS:'):
            obs_lines.append(line)
        elif line.startswith('END_PHOTOMETRY:'):
            footer.append(line)
        elif not line.startswith('TRIGGER:'): 
            header.append(line)

    if t0 is None:
        raise ValueError("Could not find PEAKMJD in the header.")

    # 2. Categorize observations into pools
    pool_less_0 = []
    pool_greater_10 = []
    
    for line in obs_lines:
        t1 = float(line.split()[1])
        delta = t1 - t0
        
        if delta < 0:
            pool_less_0.append(line)
        if delta > 10:
            pool_greater_10.append(line)

    # Make sure we have enough data to satisfy the bounds
    if not pool_less_0:
        raise ValueError("No observations found where t1 - t0 < 0.")
    if not pool_greater_10:
        raise ValueError("No observations found where t1 - t0 > 10.")

    # 3. Randomly select the first two required rows
    selected_less_0 = random.choice(pool_less_0)
    selected_greater_10 = random.choice(pool_greater_10)
    
    # Keep track of what we've already picked so we don't duplicate
    already_selected = {selected_less_0, selected_greater_10}

    # 4. Create a pool for the remaining 5 rows, excluding the ones we just picked
    pool_in_range = []
    for line in obs_lines:
        if line in already_selected:
            continue  # Skip rows we already chose
            
        t1 = float(line.split()[1])
        delta = t1 - t0
        
        if -15 <= delta <= 60:
            pool_in_range.append(line)

    if len(pool_in_range) < 5:
        raise ValueError(f"Not enough unique observations in the [-15, 60] range. Found {len(pool_in_range)}, need 5.")

    # Randomly sample exactly 5 unique rows from this range
    selected_in_range = random.sample(pool_in_range, 5)

    # 5. Combine and sort chronologically
    final_selection = [selected_less_0, selected_greater_10] + selected_in_range
    final_selection.sort(key=lambda x: float(x.split()[1]))

    # 6. Write out the new file
    with open(output_file, 'w') as f:
        for line in header:
            if line.startswith('NOBS:'):
                f.write(f"NOBS: {len(final_selection)}\n")
            else:
                f.write(line)
        
        for line in final_selection:
            f.write(line)
            
        for line in footer:
            f.write(line)

# To run it:
# filter_sn_lightcurve_random('RKCHANGEME_SDSS_SN000062.DAT', 'FILTERED_RANDOM_SN000062.DAT')

In [4]:
filter_sn_lightcurve_random('RKCHANGEME_SDSS_SN000007.DAT', 'FILTERED_RANDOM_SN000007.DAT')

In [5]:
!ls

7point_picker.ipynb	      RKCHANGEME_SDSS_SN000045.DAT
FILTERED_RANDOM_SN000007.DAT  RKCHANGEME_SDSS_SN000048.DAT
RKCHANGEME_SDSS.DUMP	      RKCHANGEME_SDSS_SN000049.DAT
RKCHANGEME_SDSS.LIST	      RKCHANGEME_SDSS_SN000050.DAT
RKCHANGEME_SDSS.README	      RKCHANGEME_SDSS_SN000052.DAT
RKCHANGEME_SDSS_SN000007.DAT  RKCHANGEME_SDSS_SN000057.DAT
RKCHANGEME_SDSS_SN000010.DAT  RKCHANGEME_SDSS_SN000059.DAT
RKCHANGEME_SDSS_SN000014.DAT  RKCHANGEME_SDSS_SN000062.DAT
RKCHANGEME_SDSS_SN000024.DAT  RKCHANGEME_SDSS_SN000063.DAT
RKCHANGEME_SDSS_SN000031.DAT  RKCHANGEME_SDSS_SN000067.DAT
RKCHANGEME_SDSS_SN000032.DAT  RKCHANGEME_SDSS_SN000070.DAT
RKCHANGEME_SDSS_SN000035.DAT  RKCHANGEME_SDSS_SN000081.DAT
RKCHANGEME_SDSS_SN000038.DAT  RKCHANGEME_SDSS_SN000083.DAT
RKCHANGEME_SDSS_SN000039.DAT  RKCHANGEME_SDSS_SN000085.DAT
RKCHANGEME_SDSS_SN000044.DAT  RKCHANGEME_SDSS_SN000098.DAT
